In [1]:
import pandas as pd
import os
from PIL import Image


In [2]:
train_dc = pd.read_csv('../data/train_data_uf.csv')
test_dc = pd.read_csv('../data/test_data_dc.csv')

In [3]:
train_dc.head()

,index,Folder,Id,Field,Well,CoringTop,CoringDown,CoringTopBind,CoringDownBind,CoreRecovery,...,PhotoDown,PhotoType,LayerTop,LayerDown,Rock,Carbonate,Ruin,Saturation,class,weight
0,1657,Unload2,1001901,Field1,Well3,2621.0,2638.0,2624.0,2641.0,16.85,...,3.91,УФ,2.63,5.34,песчаник,с карбонатными обломками или конкрециями,частично разрушен,нефтенасыщенные,1,9.367713
1,452,Unload1,1001161,Field3,Well12,3175.1,3181.1,3176.3,3182.3,5.98,...,1.45,УФ,0.90,1.45,"пересл. песчаника, алевролита и глин",с примесью,не разрушен,не опред.,0,1.369836
2,524,Unload1,1001305,Field3,Well12,3252.0,3263.0,3253.5,3264.5,10.92,...,1.00,УФ,0.00,4.04,алевролит,слабокарбонатный,не разрушен,не опред.,0,1.369836
3,609,Unload1,1001475,Field3,Well9,798.0,805.0,798.7,805.7,6.90,...,6.90,УФ,6.72,6.90,алевро-аргиллит,не карбонатный,не разрушен,не опред.,0,1.369836
4,1255,Unload2,1001097,Field8,Well14,2781.0,2799.0,2782.1,2800.1,17.99,...,16.99,УФ,12.00,17.81,кремнисто-глинистая порода,не карбонатный,не разрушен,битуминозный,2,19.707547


In [4]:
train_dc.Rock.value_counts()

песчаник                                          790
аргиллит                                          418
алевролит                                         378
пересл. песчаника, алевролита и глин              217
переслаивание песчаника и алевролита               48
кремнистая порода                                  47
песчаник глинистый                                 32
глинисто-кремнистая порода                         28
глина аргиллитоподобная                            21
карбонатно-кремнистая порода                       17
кремнисто-глинистая порода                         17
уголь                                              16
пересл. аргиллита и алевролита                     15
фундамент                                           9
алевро-аргиллит                                     8
алевролит сильноглинистый                           5
известняк                                           5
кремнисто-карбонатная порода                        4
пересл. песчаника и глины ар

In [41]:
label = train_dc.apply(lambda x: x["rock_label"]+','+x["Ruin"]+','+x["Carbonate"]+','+x["Saturation"], axis=1)


In [35]:
train_dc.Rock2 = 0 
rock_list = {"песчан":"песчаник", "аргилл":"аргиллит", "алевро":"алевролит", "пересл":"переслой", \
             "кремн":"кремнистая порода", "глин":"глина", "карбон":"карбонатная порода"}
def find_rock(inp):
    rocks = []
    for i,j in rock_list.items():
        if inp.find(i)!=-1:
            rocks.append(j)
    if len(rocks)==0: 
        rocks.append("другое")
    return rocks
rock = test_dc.Rock.apply(find_rock)

In [37]:
test_dc["rock_classes"] = rock
test_dc["rock_label"]  = rock.apply(lambda x: ','.join(x))

In [7]:
test_dc["rock_label"] = rock_label

In [38]:
test_dc.head()

,index,Folder,Id,Field,Well,CoringTop,CoringDown,CoringTopBind,CoringDownBind,CoreRecovery,...,PhotoType,LayerTop,LayerDown,Rock,Carbonate,Ruin,Saturation,class,rock_classes,rock_label
0,419,Unload1,1000981,Field7,Well13,2748.00,2757.0,0.00,0.0,8.74,...,УФ,0.00,2.47,аргиллит,не карбонатный,частично разрушен,не опред.,0,[аргиллит],аргиллит
1,307,Unload1,1000729,Field5,Well6,3142.76,3161.0,3142.26,3160.5,18.15,...,УФ,10.15,11.42,песчаник,не карбонатный,не разрушен,не опред.,0,[песчаник],песчаник
2,2454,Unload2,1003497,Field2,Well2,2747.00,2759.9,2751.00,2763.9,12.88,...,УФ,0.00,12.88,алевролит,с карбонатными обломками или конкрециями,частично разрушен,не опред.,0,[алевролит],алевролит
3,2406,Unload2,1003401,Field1,Well1,3102.00,3114.0,3104.70,3116.7,12.00,...,УФ,2.31,3.83,аргиллит,не карбонатный,частично разрушен,не опред.,0,[аргиллит],аргиллит
4,1201,Unload2,1000989,Field9,Well5,1886.00,1898.0,1884.40,1896.4,11.75,...,УФ,6.31,6.98,аргиллит,не карбонатный,разрушен,не опред.,0,[аргиллит],аргиллит


In [49]:
train_dc.rock_label = rock_label

In [12]:
train_dc.Rock = train_dc.Rock.apply(lambda x: x if x.find('пересл')==-1 else 'переслой')

In [39]:
test_dc["rock_label"].value_counts()

песчаник                                421
аргиллит                                179
алевролит                               174
песчаник,алевролит,переслой,глина       108
кремнистая порода                        28
кремнистая порода,глина                  23
песчаник,глина                           22
песчаник,алевролит,переслой              21
другое                                   18
кремнистая порода,карбонатная порода     12
аргиллит,глина                           10
алевролит,глина                           7
аргиллит,алевролит,переслой               3
песчаник,аргиллит,переслой,глина          1
аргиллит,алевролит                        1
глина                                     1
аргиллит,переслой                         1
Name: rock_label, dtype: int64

In [10]:
bad_classes = ["конгломерат", "доломит", "брекчия", "уголь","известняк","фундамент"]

In [12]:
train_dc["rock_label"] = train_dc["rock_label"].apply(lambda x: x  if x not in bad_classes else "другое")
#train_dc = train_dc.loc[temp==1,]

In [18]:
train_dc["rock_classes"] = train_dc["rock_label"].apply(lambda x: x.split(','))

In [19]:
train_dc["rock_classes"].head()

0                                [песчаник]
1    [песчаник, алевролит, переслой, глина]
2                               [алевролит]
3                     [аргиллит, алевролит]
4                [кремнистая порода, глина]
Name: rock_classes, dtype: object

In [44]:
train_uf["Saturation"].value_counts()


не опред.                   1525
нефтенасыщенные              223
битуминозный                 106
пятнисто нефтенасыщенные     100
продукт                      100
слабо нефтенасыщенные         35
Name: Saturation, dtype: int64

In [26]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
temp = mlb.fit_transform(train_dc["rock_classes"])


In [29]:
import numpy as np

In [44]:
np.sum(temp, axis = 0)

array([314, 195, 172,  18,  12,  63, 134, 573])

In [45]:
temp = pd.DataFrame(temp, columns = mlb.classes_)

In [28]:
temp = pd.read_csv('../data/train_classes.csv')

In [29]:
temp.head()

,алевролит,аргиллит,глина,другое,карбонатная порода,кремнистая порода,переслой,песчаник
0,0,0,0,0,0,0,0,1
1,1,0,1,0,0,0,1,1
2,1,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0
4,0,0,1,0,0,1,0,0


In [47]:
temp.to_csv('../data/test_classes.csv', index=False)

In [43]:
temp = mlb.transform(test_dc["rock_classes"])


In [25]:
mlb.classes_


array(['алевролит', 'аргиллит', 'глина', 'другое', 'карбонатная порода',
       'кремнистая порода', 'переслой', 'песчаник'], dtype=object)

In [15]:
import torch


tensor([0, 0, 0, 0, 0, 0, 0, 1], device='cuda:0')

In [16]:
torch.as_tensor(temp.loc[0, ], dtype = int).size()

torch.Size([8])

In [22]:
labels = []
for i in range(4):
    labels.append(torch.Tensor(temp.loc[i, ]))

In [26]:
[i for i in labels]

[tensor([0., 0., 0., 0., 0., 0., 0., 1.]),
 tensor([1., 0., 1., 0., 0., 0., 1., 1.]),
 tensor([1., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([1., 1., 0., 0., 0., 0., 0., 0.])]

In [24]:
torch.stack(labels, dim=0)

tensor([[0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.]])

In [36]:
temp.sum().index

Index(['алевролит', 'аргиллит', 'глина', 'другое', 'карбонатная порода',
       'кремнистая порода', 'переслой', 'песчаник'],
      dtype='object')

In [37]:
weight_dict = {}
for i,j in temp.sum().items():
    weight_dict[i]=j

In [38]:
weight_dict

{'алевролит': 671,
 'аргиллит': 470,
 'глина': 326,
 'другое': 33,
 'карбонатная порода': 22,
 'кремнистая порода': 116,
 'переслой': 288,
 'песчаник': 1090}

In [5]:
temp = pd.read_csv('../data/train_classes.csv')

In [6]:
temp.head()

,алевролит,аргиллит,глина,другое,карбонатная порода,кремнистая порода,переслой,песчаник
0,0,0,0,0,0,0,0,1
1,1,0,1,0,0,0,1,1
2,1,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0
4,0,0,1,0,0,1,0,0


In [8]:
temp.iloc[0, ["аргиллит","алевролит","песчаник"]]

IndexError: .iloc requires numeric indexers, got ['аргиллит' 'алевролит' 'песчаник']

In [9]:
temp.loc[0, ["аргиллит","алевролит","песчаник"]]

аргиллит     0
алевролит    0
песчаник     1
Name: 0, dtype: int64

In [10]:
len(train_dc)

2089

In [11]:
len(test_dc)

1030